In [14]:
import numpy as np
import sympy
import scipy.optimize as opt
import copy
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse import eye

In [15]:
# Parameters
rho = 0.05
r = 0.05
delta = 0.07
c_param = 2
l_param = 2
B_param = 2

params = [rho, r, delta, c_param, l_param]

# Time parameters
T = 70 # length of lifespan
nT = 280 # number of time grids
dt = T/nT # Length of each time grid

In [16]:
# Define utility functions
def u(c, c_param):
    return (c**(1-c_param))/(1-c_param)

def u_prime(c, c_param):
    return c**(-c_param)

def u_prime_inverse(x, c_param):
    return x**(-1/c_param)

def uh(h, l_param):
    return ((1-h)**(1-l_param))/(1-l_param)

def uh_prime(h, l_param):
    return -(1-h)**(-l_param)

def uh_prime_inverse(x, l_param):
    return 1 - ((-x)**(-1/l_param))

In [17]:
# Labor market functions
def g(x):
    return 1.25 - (x*(1.25**0.5 - 0.5) + 0.5)**2

def g_prime(x):
    return -2*(x*(1.25**0.5 - 0.5) + 0.5)*(1.25**0.5 - 0.5)

def g_prime_inverse(x):
    return (x/(-2*(1.25**0.5 - 0.5)) - 0.5)/(1.25**0.5 - 0.5)

Create grids

In [18]:
# State grids
na = 20 # number of asset grid points
nk = 20 # number of human capital grid points

amin = 1
amax = 10

kmin = 1
kmax = 10

a_vect = amin + np.linspace(0, 1, na)*(amax - amin) # Vector of A
da = (amax - amin)/(na - 1) # delta in A
aa = np.repeat(a_vect[:, None], nk, axis = 1) # Grid of A

k_vect = kmin + np.linspace(0, 1, nk)*(kmax - kmin) # Vector of A
dk = (kmax - kmin)/(nk - 1) # delta in K
kk = np.repeat(k_vect[None, :], na, axis = 0) # Grid of K

# Preallocate fwd/bwd derivatives, policy grid, value function grid
dVaF = np.zeros((na, nk))
dVaB = np.zeros((na, nk))
dVkF = np.zeros((na, nk))
dVkB = np.zeros((na, nk))

V_t = np.zeros((na, nk, nT+1))
C_t = np.zeros((na, nk, nT))
H_t = np.zeros((na, nk, nT))
X_t = np.zeros((na, nk, nT))
U_t = np.zeros((na, nk, nT))
mu_a_t = np.zeros((na, nk, nT))
mu_k_t = np.zeros((na, nk, nT))
dVa_t = np.zeros((na, nk, nT))
dVk_t = np.zeros((na, nk, nT))


# Arrays to store intermediate results for debugging
dVaF_t = np.zeros((na, nk, nT))
dVaB_t = np.zeros((na, nk, nT))
dVkF_t = np.zeros((na, nk, nT))
dVkB_t = np.zeros((na, nk, nT))
dVaS_t = np.zeros((na, nk, nT))
dVkS_t = np.zeros((na, nk, nT))
I_valid_t = np.zeros((na, nk, nT))
I_type_t = np.zeros((na, nk, nT))


# Terminal value
small_number1 = 10**(-16)
small_number2 = 10**(-16)
v_terminal = small_number1*((small_number2 + aa)**(1-B_param))/(1-B_param)

# Place terminal value into value function grid
V_t[:, :, nT] = v_terminal

Define functions that find dVa and dVk at boundaries a_min and k_min

In [19]:
# Drift functions
def drift_a(dVa, dVk, a, k, params):
    rho, r, delta, c_param, l_param = params
    c = u_prime_inverse(dVa, c_param)
    x = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
    h = np.clip(uh_prime_inverse(-dVa*g(x)*k - dVk*x*k, l_param), 0, 1)

    mu_a = r*a + g(x)*h*k - c
    return mu_a

def drift_k(dVk, dVa, a, k, params):
    rho, r, delta, c_param, l_param = params
    x = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
    h = np.clip(uh_prime_inverse(-dVa*g(x)*k - dVk*x*k, l_param), 0, 1)

    mu_k = a*x*h*k - delta*k
    return mu_k

In [20]:
def boundary_dVa(dVk, a, k, params):
    
    # Find dVa that leads to negative drift in a
    dVa_neg = 1
    while True:
        if drift_a(dVa_neg, dVk, a, k, params) < 0:
            break
        else:
            dVa_neg = dVa_neg/2

    # Find dVa that leads to positive drift in a
    dVa_pos = 1
    while True:
        if drift_a(dVa_pos, dVk, a, k, params) > 0:
            break
        else:
            dVa_pos = dVa_pos + 1

    # Use brentq to find the value of dVa where drift in a is 0, should be in between
    sol = opt.brentq(drift_a, dVa_neg, dVa_pos, args = (dVk, a, k, params), xtol = 1e-16, rtol = 1e-15)
    return sol

def boundary_dVk(dVa, a, k, params):
    
    # Find dVk that leads to negative drift in k
    dVk_neg = 1
    while True:
        if drift_k(dVk_neg, dVa, a, k, params) < 0:
            break
        else:
            dVk_neg = dVk_neg/2

    # Find dVk that leads to positive drift in k
    dVk_pos = 1
    while True:
        if drift_k(dVk_pos, dVa, a, k, params) > 0:
            break
        else:
            dVk_pos = dVk_pos + 1

    sol = opt.brentq(drift_k, dVk_neg, dVk_pos, args = (dVa, a, k, params), xtol = 1e-16, rtol = 1e-15)
    return sol

Find steady-state

In [21]:
# Define a function that computes mu_k for a given dVk where dVa is set to be such that mu_a=0
def drift_k_dVk(dVk, a, k, params):
    rho, r, delta, c_param, l_param = params

    # dVa that leads to mu_a = 0 for given dVk
    dVa = boundary_dVa(dVk, a, k, params)

    mu_k = drift_k(dVk, dVa, a, k, params)
    
    return mu_k

In [22]:
# Define a function that finds dVk that leads to mu_k=0 where dVa is set to be such that mu_a=0
def ss_dVk(a, k, params):
    rho, r, delta, c_param, l_param = params

    # Find dVk that leads to negative drift in k
    dVk_neg = 1
    while True:
        if drift_k_dVk(dVk_neg, a, k, params) < 0:
            break
        else:
            dVk_neg = dVk_neg/2

    # Find dVk that leads to positive drift in k
    dVk_pos = 1
    while True:
        if drift_k_dVk(dVk_pos, a, k, params) > 0:
            break
        else:
            dVk_pos = dVk_pos + 1

    sol = opt.brentq(drift_k_dVk, dVk_neg, dVk_pos, args = (a, k, params), xtol = 1e-16, rtol = 1e-15)

    return sol

In [23]:
# Define a function that computes controls and drifts for given dV and state
def control_drift(dVa, dVk, a, k, params):
    rho, r, delta, c_param, l_param = params

    c = u_prime_inverse(dVa, c_param)
    x = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
    h = np.clip(uh_prime_inverse(-dVa*g(x)*k - dVk*x*k, l_param), 0, 1)

    mu_a = r*a + g(x)*h*k - c
    mu_k = a*x*h*k - delta*k

    return c, h, x, mu_a, mu_k

In [24]:
# Find steady-state dVk0
dVk0 = np.zeros((na, nk))
dVa0 = np.zeros((na, nk))

for i in range(na):
    for j in range(nk):
        dVk0[i,j] = ss_dVk(aa[i,j], kk[i,j], params)
        dVa0[i,j] = boundary_dVa(dVk0[i,j], aa[i,j], kk[i,j], params)

In [25]:
# Fill in the rest of the steady-state values
C0, H0, X0, mu_a0, mu_k0 = control_drift(dVa0, dVk0, aa, kk, params)

Begin iterations

In [45]:
for t in range(nT-1, -1, -1):
    age = t*dt
    print(f'Age is {age}.')
    V = V_t[:, :, t+1]

    # Construct forward and backward differences for a and k
    dVaF[0:na-1, :] = (V[1:na, :] - V[0:na-1, :])/da
    dVaF[na-1, :] = dVaF[na-2, :]

    dVkF[:, 0:nk-1] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
    dVkF[:, nk-1] = dVkF[:, nk-2]

    dVaB[1:na, :] = (V[1:na, :] - V[0:na-1, :])/da
    dVaB[0, :] = dVaB[1, :]

    dVkB[:, 1:nk] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
    dVkB[:, 0] = dVkB[:, 1]

    # Verify value function is increasing in a
    if np.sum(dVaF < 0) > 0 or np.sum(dVaB < 0) > 0: # Print message if there exists an element of dVa < 0
        print('V is not monotonically increasing in a')

    if np.sum(dVkF < 0) > 0 or np.sum(dVkB < 0) > 0: # Print message if there exists an element of dVk < 0
        print('V is not monotonically increasing in k')

    # Compute policies and drift
    cFF, hFF, xFF, mu_aFF, mu_kFF = control_drift(dVaF, dVkF, aa, kk, params)
    cFB, hFB, xFB, mu_aFB, mu_kFB = control_drift(dVaF, dVkB, aa, kk, params)
    cBF, hBF, xBF, mu_aBF, mu_kBF = control_drift(dVaB, dVkF, aa, kk, params)
    cBB, hBB, xBB, mu_aBB, mu_kBB = control_drift(dVaB, dVkB, aa, kk, params)

    # Create indicator matrices indicating consistency of direction of drift with fwd/bwd difference
    # Impose False at boundary if fwd/bwd difference implies drift sends state beyond boundary
    I_FF = np.logical_and(mu_aFF > 0, mu_kFF > 0)
    I_FF[na-1, :] = False
    I_FF[:, nk-1] = False

    I_FB = np.logical_and(mu_aFB > 0, mu_kFB < 0)
    I_FB[na-1, :] = False
    I_FB[:, 0] = False

    I_BF = np.logical_and(mu_aBF < 0, mu_kBF > 0)
    I_BF[0, :] = False
    I_BF[:, nk-1] = False

    I_BB = np.logical_and(mu_aBB < 0, mu_kBB < 0)
    I_BB[0, :] = False
    I_BB[:, 0] = False

    # Steady-state indicator
    I_SS = np.zeros((na, nk), dtype=bool)

    # Compute validity of fwd/bwd difference combinations
    # Stack all indicator matrices
    I_stacked = np.zeros((na, nk, 5))
    I_stacked[:, :, 0] = I_SS
    I_stacked[:, :, 1] = I_FF
    I_stacked[:, :, 2] = I_FB
    I_stacked[:, :, 3] = I_BF
    I_stacked[:, :, 4] = I_BB

    # Use logical-or over all 4 combinations; if at least one is valid, I_valid is True
    I_valid = np.logical_or.reduce((I_SS, I_FF, I_FB, I_BF, I_BB))
    I_type = np.argmax(I_stacked, axis = 2)

    # Create matrices for boundary conditions and interior steady-states
    dVaS = np.zeros((na, nk))
    dVkS = np.zeros((na, nk))
    mu_aSF = np.zeros((na, nk))
    mu_aSB = np.zeros((na, nk))
    mu_aFS = np.zeros((na, nk))
    mu_aBS = np.zeros((na, nk))
    mu_kSF = np.zeros((na, nk))
    mu_kSB = np.zeros((na, nk))
    mu_kFS = np.zeros((na, nk))
    mu_kBS = np.zeros((na, nk))

    # Impose state boundary condition for a_min
    # Loop over a_min (upper edge) in the j dimension
    aind = 0
    aval = aa[aind, 0]
    for j in range(nk):
        if I_valid[aind, j] == False:
            kj = kk[aind, j]

            # Try SF scheme, so use dVkF
            dVkj = dVkF[aind, j]

            # Find dVaS that satisfies FOCs and a_min boundary condition
            dVaj = boundary_dVa(dVkj, aval, kj, params)

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
            cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

            # Check consistency of upwind scheme
            if mu_kj>=0 and j!=nk-1:
                # fwd diff for k is successful; SF scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 5

                # Boundary dVaS
                dVaS[aind, j] = dVaj

                # Drift values
                mu_aSF[aind, j] = mu_aj
                mu_kSF[aind, j] = mu_kj

            # SF scheme failed; try SB scheme
            else:
                dVkj = dVkB[aind, j]
                dVaj = boundary_dVa(dVkj, aval, kj, params)
                cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

                if mu_kj <= 0 and j != 0:
                    # bwd diff for k is successful; SB scheme works
                    I_valid[aind, j] = True
                    I_type[aind, j] = 6

                    # Boundary dVaS
                    dVaS[aind, j] = dVaj

                    # Drift values
                    mu_aSB[aind, j] = mu_aj
                    mu_kSB[aind, j] = mu_kj
    print('a_min completed')

    # Impose state boundary condition for a_max
    # Loop over a_max (lower edge) in the j dimension
    aind = na-1
    aval = aa[aind, 0]
    for j in range(nk):
        if I_valid[aind, j] == False:
            kj = kk[aind, j]

            # Try SF scheme, so use dVkF
            dVkj = dVkF[aind, j]

            # Find dVaS that satisfies FOCs and a_max boundary condition
            dVaj = boundary_dVa(dVkj, aval, kj, params)

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
            cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

            if mu_kj>=0 and j!=nk-1:
                # fwd diff for k is successful; SF scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 5

                # Boundary condition dVaS
                dVaS[aind, j] = dVaj

                # Drift values
                mu_aSF[aind, j] = mu_aj
                mu_kSF[aind, j] = mu_kj

            # SF scheme failed; try SB scheme
            else:
                dVkj = dVkB[aind, j]
                dVaj = boundary_dVa(dVkj, aval, kj, params)
                cstar, hstar, xstar, mu_aj, mu_kj = control_drift(dVaj, dVkj, aval, kj, params)

                if mu_kj<=0 and j!=0:
                    # bwd diff for k is successful; BB scheme works
                    I_valid[aind, j] = True
                    I_type[aind, j] = 6

                    # Boundary dVaS
                    dVaS[aind, j] = dVaj

                    # Drift values
                    mu_aSB[aind, j] = mu_aj
                    mu_kSB[aind, j] = mu_kj
    
    print('a_max completed')

    # Impose state boundary condition for k_min
    # Loop over k_min (left edge) in the i dimension
    kind = 0
    kval = kk[0, kind]
    for i in range(na):
        if I_valid[i, kind] == False:
            ai = aa[i, kind]

            # Try FS scheme, so use dVaF
            dVai = dVaF[i, kind]

            # Find dVkS that satisfies FOC and k_min boundary condition
            dVki = boundary_dVk(dVai, ai, kval, params)

            cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

            if mu_ai>=0 and i!=na:
                # fwd diff for a is successful; FS scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 7

                # Boundary condition dVkS
                dVkS[i, kind] = dVki

                # Drift values
                mu_aFS[i, kind] = mu_ai
                mu_kFS[i, kind] = mu_ki

            # FS scheme failed, try BS
            else:
                dVai = dVaB[i, kind]
                dVki = boundary_dVk(dVai, ai, kval, params)
                cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

                if mu_ai<=0 and i!=0:
                    # bwd diff for a is successful; BS scheme works
                    I_valid[i, kind] = True
                    I_type[i, kind] = 8

                    # Boundary condition dVkS
                    dVkS[i, kind] = dVki

                    # Replace bwd diff drift values
                    mu_aBS[i, kind] = mu_ai
                    mu_kBS[i, kind] = mu_ki

    print('k_min completed')

    # Impose state boundary condition for k_max
    # Loop over k_max (upper edge) in the i dimension
    kind = nk-1
    kval = kk[0, kind]
    for i in range(na):
        if I_valid[i, kind] == False:
            ai = aa[i, kind]

            # Try FS scheme, so use dVaF
            dVai = dVaF[i, kind]

            # Find dVkS that satisfies FOC and k_max boundary condition
            dVki = boundary_dVk(dVai, ai, kval, params)

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

            if mu_ai>=0 and i!=na:
                # fwd diff for a is successful; FS scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 7

                # Boundary condition dVkS
                dVkS[i, kind] = dVki

                # Drift values
                mu_aFS[i, kind] = mu_ai
                mu_kFS[i, kind] = mu_ki

            # FS scheme failed, try BS
            else:
                dVai = dVaB[i, kind]

                # Find dVkF that satisfies FOC and k_min boundary condition
                dVki = boundary_dVk(dVai, ai, kval, params)

                # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
                cstar, hstar, xstar, mu_ai, mu_ki = control_drift(dVai, dVki, ai, kval, params)

                if mu_ai<=0 and i!=0:
                    # bwd diff for a is successful; BF scheme works
                    I_valid[i, kind] = True
                    I_type[i, kind] = 8

                    # Boundary condition dVkS
                    dVkS[i, kind] = dVki

                    # Replace bwd diff drift values
                    mu_aBS[i, kind] = mu_ai
                    mu_kBS[i, kind] = mu_ki

    print('k_max completed')

    
    # Look for a or k steady-states in the interior
    for i in range(1, na-1):
        for j in range(1, nk-1):
            if I_valid[i, j] == False:

                aij = aa[i, j]
                kij = kk[i, j]

                dVaijF = dVaF[i, j]
                dVaijB = dVaB[i, j]
                dVkijF = dVkF[i, j]
                dVkijB = dVkB[i, j]

                # Check consistency
                mu_aijFF = mu_aFF[i, j]
                mu_kijFF = mu_kFF[i, j]

                mu_aijFB = mu_aFB[i, j]
                mu_kijFB = mu_kFB[i, j]

                mu_aijBF = mu_aBF[i, j]
                mu_kijBF = mu_kBF[i, j]

                mu_aijBB = mu_aBB[i, j]
                mu_kijBB = mu_kBB[i, j]

                aF_consistent = ((mu_aijFF > 0) and (mu_aijFB > 0))
                aB_consistent = ((mu_aijBF < 0) and (mu_aijBB < 0))
                kF_consistent = ((mu_kijFF > 0) and (mu_kijBF > 0))
                kB_consistent = ((mu_kijFB < 0) and (mu_kijBB < 0))

                a_consistent = (aF_consistent or aB_consistent)
                k_consistent = (kF_consistent or kB_consistent)

                # k is steady-state
                if (a_consistent == True) and (k_consistent == False):
                    print('k is steady-state')

                    # k is steady-state, so need to find dVk such that mu_k = 0
                    # First try FS scheme
                    dVaij = dVaF[i, j]

                    # Find dVk that leads to mu_k = 0
                    dVkij = boundary_dVk(dVaij, aij, kij, params)

                    # Policy and drift
                    cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                    # If FS is consistent:
                    if mu_aij > 0:
                        dVkS[i, j] = dVkij
                        mu_aFS[i, j] = mu_aij
                        mu_kFS[i, j] = mu_kij

                        I_valid[i, j] = True
                        I_type[i, j] = 7

                    # FS scheme failed; try BS scheme
                    else:
                        dVaij = dVaB[i, j]

                        # Find dVk that leads to mu_k = 0
                        dVkij = boundary_dVk(dVaij, aij, kij, params)

                        # Policy and drift
                        cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                        # If BS is consistent:
                        if mu_aij < 0:
                            dVkS[i, j] = dVkij
                            mu_aBS[i, j] = mu_aij
                            mu_kBS[i, j] = mu_kij

                            I_valid[i, j] = True
                            I_type[i, j] = 8


                if (a_consistent == False) and (k_consistent == True):
                    print('a is steady-state')

                    # a is steady-state, so need to find dVa such that mu_a = 0
                    # First try SF scheme
                    dVkij = dVkF[i, j]

                    # Find dVk that leads to mu_k = 0
                    dVaij = boundary_dVa(dVkij, aij, kij, params)

                    # Policy and drift
                    cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                    # If SF is consistent:
                    if mu_kij > 0:
                        dVaS[i, j] = dVaij
                        mu_aSF[i, j] = mu_aij
                        mu_kSF[i, j] = mu_kij

                        I_valid[i, j] = True
                        I_type[i, j] = 5

                    # SF scheme failed; try SB scheme
                    else:
                        dVkij = dVkB[i, j]

                        # Find dVk that leads to mu_k = 0
                        dVaij = boundary_dVa(dVkij, aij, kij, params)

                        # Policy and drift
                        cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                        # If BS is consistent:
                        if mu_kij < 0:
                            dVaS[i, j] = dVaij
                            mu_aSB[i, j] = mu_aij
                            mu_kSB[i, j] = mu_kij

                            I_valid[i, j] = True
                            I_type[i, j] = 6

                if (a_consistent == False) and (k_consistent == False):
                    print('a and k are steady-state')


    
    # Construct dVa and dVk matrices
    # If no scheme is valid, use steady-state values, i.e. stay put
    # Note: SS is I_type == 0, FF == 1, FB == 2, BF == 3, BB == 4, SF == 5, SB == 6, FS == 7, BS == 8 
    dVa = (dVaF*(I_type == 1) + dVaF*(I_type == 2) + dVaB*(I_type == 3) + dVaB*(I_type == 4) + dVaS*(I_type == 5) + dVaS*(I_type == 6) + dVaF*(I_type == 7) + dVaB*(I_type == 8))*I_valid + dVa0*(~I_valid)
    dVk = (dVkF*(I_type == 1) + dVkB*(I_type == 2) + dVkF*(I_type == 3) + dVkB*(I_type == 4) + dVkF*(I_type == 5) + dVkB*(I_type == 6) + dVkS*(I_type == 7) + dVkS*(I_type == 8))*I_valid + dVk0*(~I_valid)

    # Construct policy functions and utility
    C, H, X, mu_a, mu_k = control_drift(dVa, dVk, aa, kk, params)
    U = u(C, c_param) + uh(H, l_param)

    # Construct sparse diagonal matrix A
    # Reshape mu_a using column-major (Fortran) order
    mu_a_diag = np.reshape(mu_a, na*nk, order='F')/da

    # -1 diagonal is mu_a<0, +1 diag is mu>0
    # Shift elements forward 1 index as placement in upper diagonal in spdiags lops off the first element. Want diagonal to start with the 1st element.
    mu_a_diag_pos = np.roll(np.clip(mu_a_diag, 0, None), 1)

    # Shift elements back 1 index as placement in lower diagonal in spdiags lops off the last element. Want diagonal to start with the 2nd element.
    mu_a_diag_neg = np.roll(-np.clip(mu_a_diag, None, 0), -1) 

    # Reshape mu_k using column-major (Fortran) order
    mu_k_diag = np.reshape(mu_k, na*nk, order='F')/dk

    # -na diag is mu_k < 0, +na diag is mu_k > 0
    # Shift elements forward na index as placement in upper diagonal in spdiags lops off the first na elements. Want diagonal to start with 1st element.
    mu_k_diag_pos = np.roll(np.clip(mu_k_diag, 0, None), na)

    # Shift elements back na index as placement in lower diagonal in spdiags lops off the last na elements. Want diagonal to start with the na'th element
    mu_k_diag_neg = np.roll(-np.clip(mu_k_diag, None, 0), -na)

    # Construct main diagonal
    main_diag = np.reshape(-np.abs(mu_a)/da - np.abs(mu_k)/dk, na*nk, order='F')

    A = spdiags(np.array([mu_k_diag_neg, mu_a_diag_neg, main_diag, mu_a_diag_pos, mu_k_diag_pos]), np.array([-na, -1, 0, 1, na]), na*nk, na*nk)

    # Solve set of linear equations to obtain V for current period from V of next period
    b = np.reshape(U + V/dt, na*nk, order='F')
    B = (1/dt + rho)*np.identity(na*nk) - A
    V_stacked = np.linalg.solve(B, b)

    V_t[:, :, t] = copy.copy(np.reshape(V_stacked, (na, nk), order = 'F'))

    # Store results for debugging
    C_t[:, :, t] = copy.copy(C)
    H_t[:, :, t] = copy.copy(H)
    X_t[:, :, t] = copy.copy(X)
    U_t[:, :, t] = copy.copy(U)

    mu_a_t[:, :, t] = copy.copy(mu_a)
    mu_k_t[:, :, t] = copy.copy(mu_k)

    dVaF_t[:, :, t] = copy.copy(dVaF)
    dVaB_t[:, :, t] = copy.copy(dVaB)
    dVaS_t[:, :, t] = copy.copy(dVaS)
    dVkF_t[:, :, t] = copy.copy(dVkF)
    dVkB_t[:, :, t] = copy.copy(dVkB)
    dVkS_t[:, :, t] = copy.copy(dVkS)

    dVa_t[:, :, t] = copy.copy(dVa)
    dVk_t[:, :, t] = copy.copy(dVk)

    I_valid_t[:, :, t] = copy.copy(I_valid)
    I_type_t[:, :, t] = copy.copy(I_type)

Age is 69.75.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 69.5.
a_min completed
a_max completed
k_min completed
k_max completed
Age is 69.25.
a_min completed
a_max completed
k_min completed
k_max completed
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
Age is 69.0.
V is not monotonically increasing in a
a_min completed
a_max completed
k_min completed
k_max completed
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state


KeyboardInterrupt: 

In [34]:
t = nT-3

ai = aa[i, j]
kj = kk[i, j]

dVaF = dVaF_t[:, :, t]
dVaB = dVaB_t[:, :, t]
dVkF = dVkF_t[:, :, t]
dVkB = dVkB_t[:, :, t]

I_valid = I_valid_t[:, :, t]
I_type = I_type_t[:, :, t]

In [35]:
# Look for a or k steady-states in the interior
for i in range(1, na-1):
    for j in range(1, nk-1):
        if I_valid[i, j] == False:

            aij = aa[i, j]
            kij = kk[i, j]

            dVaijF = dVaF[i, j]
            dVaijB = dVaB[i, j]
            dVkijF = dVkF[i, j]
            dVkijB = dVkB[i, j]

            # Check consistency
            mu_aijFF = mu_aFF[i, j]
            mu_kijFF = mu_kFF[i, j]

            mu_aijFB = mu_aFB[i, j]
            mu_kijFB = mu_kFB[i, j]

            mu_aijBF = mu_aBF[i, j]
            mu_kijBF = mu_kBF[i, j]

            mu_aijBB = mu_aBB[i, j]
            mu_kijBB = mu_kBB[i, j]

            aF_consistent = ((mu_aijFF > 0) and (mu_aijFB > 0))
            aB_consistent = ((mu_aijBF < 0) and (mu_aijBB < 0))
            kF_consistent = ((mu_kijFF > 0) and (mu_kijBF > 0))
            kB_consistent = ((mu_kijFB < 0) and (mu_kijBB < 0))

            a_consistent = (aF_consistent or aB_consistent)
            k_consistent = (kF_consistent or kB_consistent)

            # k is steady-state
            if (a_consistent == True) and (k_consistent == False):
                print('k is steady-state')

                # k is steady-state, so need to find dVk such that mu_k = 0
                # First try FS scheme
                dVaij = dVaF[i, j]

                # Find dVk that leads to mu_k = 0
                dVkij = boundary_dVk(dVaij, aij, kij, params)

                # Policy and drift
                cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                # If FS is consistent:
                if mu_aij > 0:
                    dVkS[i, j] = dVkij
                    mu_aFS[i, j] = mu_aij
                    mu_kFS[i, j] = mu_kij

                    I_valid[i, j] = True
                    I_type[i, j] = 7

                # FS scheme failed; try BS scheme
                else:
                    dVaij = dVaB[i, j]

                    # Find dVk that leads to mu_k = 0
                    dVkij = boundary_dVk(dVaij, aij, kij, params)

                    # Policy and drift
                    cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                    # If BS is consistent:
                    if mu_aij < 0:
                        dVkS[i, j] = dVkij
                        mu_aBS[i, j] = mu_aij
                        mu_kBS[i, j] = mu_kij

                        I_valid[i, j] = True
                        I_type[i, j] = 8


            if (a_consistent == False) and (k_consistent == True):
                print('a is steady-state')

                # a is steady-state, so need to find dVa such that mu_a = 0
                # First try SF scheme
                dVkij = dVkF[i, j]

                # Find dVk that leads to mu_k = 0
                dVaij = boundary_dVa(dVkij, aij, kij, params)

                # Policy and drift
                cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                # If SF is consistent:
                if mu_kij > 0:
                    dVaS[i, j] = dVaij
                    mu_aSF[i, j] = mu_aij
                    mu_kSF[i, j] = mu_kij

                    I_valid[i, j] = True
                    I_type[i, j] = 5

                # SF scheme failed; try SB scheme
                else:
                    dVkij = dVkB[i, j]

                    # Find dVk that leads to mu_k = 0
                    dVaij = boundary_dVa(dVkij, aij, kij, params)

                    # Policy and drift
                    cij, hij, xij, mu_aij, mu_kij = control_drift(dVaij, dVkij, aij, kij, params)

                    # If BS is consistent:
                    if mu_kij < 0:
                        dVaS[i, j] = dVaij
                        mu_aSB[i, j] = mu_aij
                        mu_kSB[i, j] = mu_kij

                        I_valid[i, j] = True
                        I_type[i, j] = 6

            if (a_consistent == False) and (k_consistent == False):
                print('a and k are steady-state')

k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state
k is steady-state


In [37]:
I_type

array([[0., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6., 6.],
       [8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [8., 8., 4., 4., 4., 4.

In [41]:
mu_a_t[:, :, t]

array([[-1.11022302e-16, -1.11022302e-16,  2.22044605e-16,
         1.11022302e-16,  0.00000000e+00,  2.22044605e-16,
         0.00000000e+00, -4.44089210e-16, -2.22044605e-16,
         2.22044605e-16,  6.66133815e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  4.44089210e-16,
        -8.88178420e-16,  4.44089210e-16, -4.44089210e-16,
        -4.44089210e-16, -8.88178420e-16],
       [-4.67064124e+01, -5.77095297e+01, -6.54985126e+01,
        -7.22403287e+01, -7.82367371e+01, -8.36697433e+01,
        -8.86587879e+01, -9.32870677e+01, -9.76151745e+01,
        -1.01688791e+02, -1.05543330e+02, -1.09206871e+02,
        -1.12702109e+02, -1.16047677e+02, -1.19259095e+02,
        -1.22349438e+02, -1.25329842e+02, -1.28209879e+02,
        -1.30997844e+02, -1.33700980e+02],
       [-5.05903637e+01, -1.11022302e-16, -7.11782845e+01,
        -7.85059973e+01, -8.50235063e+01, -9.09286395e+01,
        -9.63512213e+01, -1.01381682e+02, -1.06085879e+02,
        -1.10513468e+02, -1.1

In [44]:
mu_kBS

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.94289029e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.0